In [1]:
import time
import numpy as np
from itertools import product
from matplotlib import pyplot as plt
from matplotlib import cm

In [2]:
############## Input Script #############

N = int(1e4)  # approximate number of particles
Lx, Ly = 64, 64  # size of the system
Nx, Ny = 64, 64  # number of grid points
dx, dy = Lx / Nx, Ly / Ny  # delta_x and delta_y
Bx, By, Bz = 0.0, 0.0, 0.0  # external magnetic field
B = np.array([Bx, By, Bz])

vd, vt = 5.0, 1.0  # drift and thermal velocities

steps = 200  # total of steps
dt = 0.1  # time step

output = "electrosctatic"  # output folder

margin = dx / 10  # safety margin so the particles don't initialize in the border

line_positions = np.linspace(margin, Lx - margin, int(np.sqrt(N)))

positions = np.array(list(product(line_positions, repeat=2)))

np.random.shuffle(positions)
vel_zero = np.zeros(int(N / 2))
vel_left = np.random.normal(-vd, vt, size=int(N / 4))
vel_right = np.random.normal(vd, vt, size=int(N / 4))
Vx = np.array([np.concatenate((vel_left, vel_right, vel_zero))])
Vy = np.array([np.zeros(N)])
Vz = np.array([np.zeros(N)])
velocities = np.concatenate((Vx.T, Vy.T, Vz.T),axis=1)
charges = np.concatenate((-np.ones(int(N / 2)), np.ones(int(N / 2))))
QoverM = np.concatenate((-np.ones(int(N / 2)), np.ones(int(N / 2))))
moves = np.concatenate((np.ones(int(N / 2)), np.zeros(int(N / 2)))) # ions are fixed
N = np.min([positions.shape[0], velocities.shape[0], charges.shape[0], QoverM.shape[0], moves.shape[0]])

In [3]:
def density(positions, charges, dx, dy, Nx, Ny, N):
    """
    density(): returns the charge density on each node of the system.
    
    Input:
        positions:  Position vector of each particle.        
        charges:    Electric charge of each particle.  
        dx, dy:     Cell size.                         
        Nx, Ny:     Grid size of the system.          
        N:          Number of particles.              
    
    Output:
        rho:        Charge density on each node.       
    """
    
    rho = np.zeros([Nx, Ny])
    
    for p in range(N):
        i = int(np.floor(positions[p][0]/dx))
        j = int(np.floor(positions[p][1]/dy))
        hx = positions[p][0] - (i * dx)
        hy = positions[p][1] - (j * dy)
        nxt_i = np.mod(i+1, Nx)
        nxt_j = np.mod(j+1, Ny)
        
        rho[i][j] += charges[p] * (dx - hx) * (dy - hy)
        rho[i][nxt_j] += charges[p] * (dx - hx) * hy
        rho[nxt_i][j] += charges[p] * hx * (dy - hy)
        rho[nxt_i][nxt_j] += charges[p] * hx * hy
    
    rho /= (dx * dx * dy * dy)
    
    return rho

In [4]:
def potential(rho, dx, dy, Nx, Ny):
    """
    potential(): returns the electric potential on each node of the system
    
    Input:
        rho:        Charge density on each node.      
        dx, dy:     Cell size.                       
        Nx, Ny:     Grid size of the system.         
    
    Output:
        phi:        Electric potential on each node.   
    """
    
    rho_k = np.fft.fft2(rho)
    phi_k = rho_k
    
    Wx = np.exp(1j*2*np.pi/Nx)
    Wy = np.exp(1j*2*np.pi/Ny)
    Wn = 1.0
    Wm = 1.0
    dx_2 = dx * dx
    dy_2 = dy * dy
    
    for n in range(Nx):
        for m in range(Ny):
            denom = dy_2 * (2.0 - Wn - 1.0 / Wn) + dx_2 * (2.0 - Wm - 1.0 / Wm)
            if (denom != 0.0):
                phi_k[n][m] *= (dx_2 * dy_2) / denom
            Wm *= Wy
        Wn *= Wx
    
    phi = np.fft.ifft2(phi_k)
    phi = np.real(phi)
    
    return phi

In [5]:
def fieldNodes(phi, dx, dy, Nx, Ny):
    """
    fieldNodes(): returns the electric field components x, y, and z on each node of the system
    
    Input:
        phi:        Electric potential on each node.   
        dx, dy:     Cell size.                         
        Nx, Ny:     Grid size of the system.           
    
    Output:
        field:          Electric field acting on each particle. 
    """
    field = np.zeros([Nx, Ny, 3])
    
    for j in range(Ny):
        for i in range(Nx):
            nxt_i = np.mod(i+1, Nx)
            prv_i = np.mod(i-1, Nx)
            field[i][j][0] = (phi[prv_i][j] - phi[nxt_i][j]) / (dx * 2.0)
            
    for i in range(Nx):
        for j in range(Ny):
            nxt_j = np.mod(j+1, Ny)
            prv_j = np.mod(j-1, Ny)
            field[i][j][1] = (phi[i][prv_j] - phi[i][nxt_j]) / (dx * 2.0)
    
    return field

In [6]:
def fieldParticles(field, positions, moves, dx, dy, Nx, Ny, N):
    """
    fieldParticles(): returns the electric field components x, y, and z acting on each particle
    
    Input:
        field:      Electric field on each node.       
        positions:  Position vector of each particle.        
        moves:      Indexes of moving particles.      
        dx, dy:     Cell size.                       
        Nx, Ny:     Grid size of the system.          
        N:          Number of particles.               
    
    Output:
        E:          Electric field acting on each particle. 
    """
    
    E = np.zeros([N, 3])
    
    for p in range(N):
        if (moves[p]):
            i = int(np.floor(positions[p][0]/dx))
            j = int(np.floor(positions[p][1]/dy))
            hx = positions[p][0] - (i * dx)
            hy = positions[p][1] - (j * dy)
            nxt_i = np.mod(i+1, Nx)
            nxt_j = np.mod(j+1, Ny)
            
            A = (dx - hx) * (dy - hy)
            B = (dx - hx) * hy
            C = hx * (dy - hy)
            D = hx * hy
        
            E[p][0] = field[i][j][0] * A + field[i][nxt_j][0] * B + field[nxt_i][j][0] * C + field[nxt_i][nxt_j][0] * D
            E[p][1] = field[i][j][1] * A + field[i][nxt_j][1] * B + field[nxt_i][j][1] * C + field[nxt_i][nxt_j][1] * D
            
    E /= (dx * dy)
    
    return E

In [7]:
def boris(velocities, QoverM, moves, E, B, dt, N):
    """
    boris(): performs the Boris algorithm on the velocities of the particles
    
    Input:
        velocities: Velocities vector of each particle.         
        QoverM:     q/m relation of each particle.     
        moves:      Indexes of moving particles.      
        E:          Electric field acting on each particle. 
        B:          Uniform external magnetic field. 
        dt:         Time step.                       
        N:          Number of particles.              
    
    Output:
        velocities: Velocities vector of each particle after boris pusher.
    """
    
    for p in range(N):
        if (moves[p]):
            t = 0.5 * QoverM[p] * B * dt;
            norm_t = np.linalg.norm(t)
            t_2 = norm_t * norm_t
            s = (2.0 * t) / (1.0 + t_2)
            v_minus = velocities[p] + 0.5 * QoverM[p] * E[p] * dt
            v_prime = v_minus + np.cross(v_minus, t)
            v_plus = v_minus + np.cross(v_prime, s)
            velocities[p] = v_plus + 0.5 * QoverM[p] * E[p] * dt
    
    return velocities

In [8]:
def update(positions, velocities, moves, Lx, Ly, N):
    """
    update(): updates the velocity and positions of the particles
    
    Input:
        positions:  Position vector of each particle.         
        velocities: Velocities vector of each particle.  
        moves:      Indexes of moving particles.      
        Lx, Ly:  Length of the system.  
        N:          Number of particles.              
    
    Output:
        positions:  Position vector of each particle after boris pusher.
    """
    
    for p in range(N):
        if (moves[p]):
            positions[p][0] += velocities[p][0] * dt
            positions[p][1] += velocities[p][1] * dt
            positions[p][0] = np.mod(positions[p][0], Lx);
            positions[p][1] = np.mod(positions[p][1], Ly);
    
    return positions

In [9]:
import time
start = time.time()

for step in range(steps):
    rho = density(positions, charges, dx, dy, Nx, Ny, N);
    phi = potential(rho, dx, dy, Nx, Ny);
    field = fieldNodes(phi, dx, dy, Nx, Ny);
    E = fieldParticles(field, positions, moves, dx, dy, Nx, Ny, N);
    velocities  = boris(velocities, QoverM, moves, E, B, dt, N)
    positions = update(positions, velocities, moves, Lx, Ly, N)
    
    # plot potential
    plt.figure(figsize=(10,10))
    plt.title(r"$\omega_{\rm{pe}}$ "+str(step)+" timesteps", fontsize=25)
    psm = plt.imshow(phi, cmap=cm.get_cmap('jet'), vmin=-30, vmax=30)
    plt.colorbar(psm)
    plt.xlabel(r"$x / \lambda_D$", fontsize=25)
    plt.ylabel(r"$y / \lambda_D$", fontsize=25)
    plt.savefig('C://Users//admin//data//H{}'.format(step)+'.png')
    plt.close()
    
    # plot phase space
    plt.figure(figsize=(10,10))
    plt.title(r"$\omega_{\rm{pe}}$ "+str(step)+" timesteps", fontsize=25)
    plt.scatter(positions[:int(N/4),0], velocities[:int(N/4),0], alpha=0.9)
    plt.scatter(positions[int(N/4+1):int(N/2),0], velocities[int(N/4+1):int(N/2),0], alpha=0.9)
    plt.xlabel(r"$x / \lambda_D$", fontsize=25)
    plt.ylabel(r"$v_x / v_{\rm{th}}$", fontsize=25)
    plt.savefig('C://Users//admin//data//S{}'.format(step)+'.png')
    plt.close()
    
end = time.time()
interval = end - start
print(np.floor(interval/60)," minutes ",interval%60," seconds ")

6.0  minutes  56.74530267715454  seconds 


In [10]:
import os
import imageio
import warnings
warnings.filterwarnings("ignore")

# plot potential
path = "C:/Users/admin/data/H"
image_list = [path + str(x) + ".png" for x in range(steps)]
gif_name = 'potential.gif'

frames = []
for image_name in image_list:
    frames.append(imageio.imread(image_name))

imageio.mimsave(gif_name, frames, 'GIF', duration=0.1)

# plot phase space
path = "C:/Users/admin/data/S"
image_list = [path + str(x) + ".png" for x in range(steps)]
gif_name = 'phase_space.gif'

frames = []
for image_name in image_list:
    frames.append(imageio.imread(image_name))

imageio.mimsave(gif_name, frames, 'GIF', duration=0.1)

# remove pictures
file_name = "C:/Users/admin/data"
for root, dirs, files in os.walk(file_name):
    for name in files:
        if name.endswith(".png"):
            os.remove(os.path.join(root, name))
           
print('finished')

finished
